# Pre-process input data for coastal variable extraction

Author: Emily Sturdivant; esturdivant@usgs.gov

***

## Basic setup
We recommend that you install this package in the ArcGIS Pro conda environment through pip: 
```
\ArcGIS\Pro\bin\Python\Scripts\proenv
pip install git+https://github.com/esturdivant-usgs/BI-geomorph-extraction.git
```

You must run this notebook within the ArcGIS Pro conda environment. To do so, type the following in your command prompt (assuming it has the default set-up and substituting path\to\dir with the location of the repository):

```bat
cd path\to\dir\BI-geomorph-extraction
\ArcGIS\Pro\bin\Python\Scripts\proenv
jupyter notebook
```

## Pre-processing steps

1. Pre-created geomorphic features: dunes, shoreline points, armoring.
2. Inlets
3. Shoreline
4. Transects - extend and sort
5. Transects - tidy

#### Import modules

Sometimes the user needs to run the code through the ArcPy console (when it involves selecting layers). --- an alternative may be to use the Pro version of the mapping toolbox and say current map document... --- In this case, we need to make sure that ArcGIS can find this package. 

In [1]:
import os
import sys
import time
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import arcpy
if os.path.basename(os.getcwd()) == 'CoastalVarExtractor':
    print('Module path is: {}'.format(os.getcwd()))
    import CoastalVarExtractor.functions_warcpy as fwa

Module path is: C:\Users\esturdivant\Code\BI-geomorph-extraction\CoastalVarExtractor


If you need to run the code inside the ArcPy console... 

In [ ]:
mod_path = r"C:\Users\esturdivant\Code\BI-geomorph-extraction" # replace with path to module
sys.path.insert(0, mod_path)
import CoastalVarExtractor.functions_warcpy as fwa

## Initialize variables

Based on the project directory, and the site and year you have input, setvars.py will set a bunch of variables as the names of folders, files, and fields. 1) set-up the project folder and paths: 

In [2]:
from CoastalVarExtractor.setvars import *
from CoastalVarExtractor.configmap import *
proj_dir = r"\\Mac\stor\Projects\TransectExtraction\Fisherman2014"
site = "Fisherman"
year = "2014"
SiteYear_strings = siteyear[site+year] # get siteyear dict from configmap
home = os.path.join(proj_dir, '{site}{year}.gdb'.format(**SiteYear_strings))
arcpy.env.workspace = home
arcpy.env.scratchWorkspace = proj_dir

setvars.py initialized variables.


In [3]:
extendedTrans = 'extTrans'
inletLines = 'inletLines'
ShorelinePts = 'SLpts'
dlPts = 'DLpts'
dhPts = 'DHpts'
armorLines = 'armorLines'
elevGrid = 'DEM'
elevGrid_5m = 'DEM_5m'

extendedTrans = os.path.join(home, 'extTrans')
inletLines = os.path.join(home, 'inletLines')
ShorelinePts = os.path.join(home, 'SLpts')
dlPts = os.path.join(home, 'DLpts')
dhPts = os.path.join(home, 'DHpts')
armorLines = os.path.join(home, 'armorLines')
elevGrid = os.path.join(home, 'DEM')
elevGrid_5m = os.path.join(home, 'DEM_5m')

## Dunes and armoring <a name="geofeatures"></a>
Display the points and the DEM in a GIS to check for irregularities. For example, if shoreline points representing a distance less than X m are visually offset from the general shoreline, they should likely be removed. Another red flag is when the positions of dlows and dhighs in relation to the shore are illogical, i.e. dune crests are seaward of dune toes. 

#### Replace fill values with Null. 

In [ ]:
fwa.ReplaceValueInFC(dhPts, oldvalue=fill, newvalue=None, fields=["dhigh_z"])      # Dhighs
fwa.ReplaceValueInFC(dlPts, oldvalue=fill, newvalue=None, fields=["dlow_z"])       # Dlows
fwa.ReplaceValueInFC(ShorelinePts, oldvalue=fill, newvalue=None, fields=["slope"]) # Shoreline

#### Armoring
If the dlows do not capture the entire top-of-beach due to atypical formations caused by anthropogenic modification, you may need to digitize the beachfront armoring. The next code block will generate an empty feature class. Refer to the DEM and orthoimagery. If there is no armoring in the study area, continue. If there is armoring, use the Editing toolbar to add lines to the feature class that trace instances of armoring. Common manifestations of what we call armoring are sandfencing and sandbagging and concrete seawalls. 

If there is no armoring file in the project geodatabase, the extractor script will notify you that it is proceeding without armoring.

*__Requires manipulation in GIS__*

In [ ]:
arcpy.CreateFeatureclass_management(home, armorLines, 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now manually digitize the shorefront armoring.".format(armorLines))

## Inlets
We also need to manually digitize inlets if an inlet delineation does not already exist. To do, the code will produce the feature class. After which use the Editing toolbar to create a line when the island shore meets a tidal inlet. If the study area includes both sides of an inlet, that inlet will be represented by two lines. The inlet lines are use to define the bounds of the oceanside shore, which is also considered the point where the oceanside shore meets the bayside. Inlet lines must intersect the MHW contour. 

*__Requires manipulation in GIS__*

In [ ]:
# manually create lines that correspond to end of land and cross the MHW line (use bndpoly/DEM)
arcpy.CreateFeatureclass_management(home, inletLines, 'POLYLINE', spatial_reference=utmSR)
print("{} created. Now we'll stop for you to manually create lines at each inlet.".format(inletLines))

## Shoreline
The shoreline is produced through a combination of the DEM and the shoreline points. The first step converts the DEM to both MTL and MHW contour polygons. Those polygons are combined to produce the full shoreline, which is considered to fall at MHW on the oceanside and MTL on the bayside (to include partially submerged wetland). User input is required to identify only the areas within the study area and eliminate isolated landmasses that are not. 

In [ ]:
bndpoly = fwa.DEMtoFullShorelinePoly(elevGrid_5m, MTL, MHW, inletLines, ShorelinePts)
print('Select features from {} that should not be included in {}'.format(bndpoly, barrierBoundary))

In [ ]:
bndMTL = 'bndpoly_mtl'
bndMHW = 'bndpoly_mhw'
bndpoly = 'bndpoly'
print("Creating the MTL contour polgon from the DEM...")
RasterToLandPerimeter(elevGrid, bndMTL, MTL)  # Polygon of MTL contour
print("Creating the MHW contour polgon from the DEM...")
RasterToLandPerimeter(elevGrid, bndMHW, MHW)  # Polygon of MHW contour

*__Requires display in GIS__*

In [ ]:
print("Combining the two polygons...")
bndpoly = fwa.CombineShorelinePolygons(bndMTL, bndMHW, inletLines, ShorelinePts, bndpoly)

Once the features to delete are selected, either delete in the GIS or run the code.
__Do not:__ Select the features in ArcGIS and then run DeleteFeatures from here. That will delete the entire feature class. 

In [ ]:
# Change bndpoly to match the layer name before running.
bndpoly = bndpoly
arcpy.DeleteFeatures_management(bndpoly)

The next step snaps the boundary polygon to the shoreline points anywhere they don't already match and as long as as they are within 25 m of each other. 

In [ ]:
bndpoly = 'bndpoly'
barrierBoundary = fwa.NewBNDpoly(bndpoly, ShorelinePts, barrierBoundary, '25 METERS', '50 METERS')
print("Created: '{}'".format(barrierBoundary))

In [ ]:
shoreline = fwa.CreateShoreBetweenInlets(barrierBoundary, inletLines, shoreline, 
                                         ShorelinePts, proj_code)

After this step, you'll want to make sure the shoreline looks okay. There should be only one line segment for each stretch of shore between two inlets. Segments may be incorrectly deleted if the shoreline points are missing in the area. Segments may be incorrectly preserved if they are intersect a shoreline point. To rectify, either perform manual editing or rerun this code with modifications. 

## Transects
### Extended transects

Create extendedTrans, NASC transects for the study area extended to cover the island, with gaps filled, and sorted in the field sort_ID.

#### 1. Extend and Copy only the geometry of transects to use as material for filling gaps

In [4]:
orig_trans = os.path.join(arcpy.env.workspace, 'DelmarvaS_SVA_LT')

('\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\Fisherman2014.gdb',
 'DelmarvaS_SVA_LT')

In [7]:
# Delete transects over 200 m outside of the study area.
if input("Need to remove extra transects? 'y' if barrierBoundary exists and should be used to select. ") == 'y':
    fwa.RemoveTransectsOutsideBounds(orig_trans, barrierBoundary)
fwa.ExtendLine(orig_trans, trans_extended, extendlength, proj_code)
fwa.CopyAndWipeFC(trans_extended, trans_presort, ['sort_ID'])
print("MANUALLY: use groups of existing transects in new FC '{}' to fill gaps.".format(trans_presort))

Need to remove extra transects? 'y' if barrierBoundary exists and should be used to select. y
\\Mac\stor\Projects\TransectExtraction\Fisherman2014\Fisherman2014.gdb\DelmarvaS_SVA_LT is already projected in UTM.
MANUALLY: use groups of existing transects in new FC '\\Mac\stor\Projects\TransectExtraction\Fisherman2014\scratch.gdb\trans_presort_temp' to fill gaps.


*__Requires manipulation in GIS__*

1. Edit the trans_presort_temp feature class. __Move and rotate__ groups of transects to fill in gaps that are greater than 50 m alongshore. There is no need to preserve the original transects, but avoid overlapping the transects with each other and with the originals. Do not move any transects slightly. If they are moved, they will not be deleted in the next stage. If you slightly move any, you can either undo or delete that line entirely.

In [8]:
fwa.RemoveDuplicates(trans_presort, trans_extended, barrierBoundary)

'\\\\Mac\\stor\\Projects\\TransectExtraction\\Fisherman2014\\scratch.gdb\\trans_presort_temp'

#### 2. Sort the transects.

First set the sorting parameters. If the shoreline curves, the GIS will not correctly establish the alongshore order by simple ordering from the identified sort_corner so you need to identify different groups of transects for sorting. If this is the case, answer yes to the next prompt.

In [6]:
sort_lines = fwa.SortTransectPrep(spatialref=utmSR)

Do we need to sort the transects in batches to preserve the order? (y/n) y
MANUALLY: Add features to sort_lines. Indicate the order of use in 'sort' and the sort corner in 'sort_corn'.


*__Requires manipulation in GIS__*

The last step generated an empty sort lines feature class if you indicated that transects need to be sorted in batches to preserve the order. Now, the user creates lines that will be used to spatially sort transects in groups. 

For each group of transects:

1. __Create a new line__ in 'sort_lines' that intersects all transects in the group. The transects intersected by the line will be sorted independently before being appended to the preceding groups.  (*__add example figure__*)
2. __Assign values__ for the fields 'sort' and 'sort_corner.' 'sort' indicates the order in which the line should be used and 'sort_corn' indicates the corner from which to perform the spatial sort ('LL', 'UL', etc.).
3. Run the following code to create a new sorted transect file.


In [ ]:
fwa.SortTransectsFromSortLines(trans_presort, extendedTrans, sort_lines, tID_fld, sort_corner=sort_corner)

### Tidy the extended transects

*__Requires manipulation in GIS__*

Overlapping transects cause problems during conversion to 5-m points and to rasters. We create a separate feature class with the 'tidied' transects, in which the lines don't overlap. This is largely a manually process with the following steps: 

1. __Select__ transects to be used to split other transects. Prioritize transects that were originally from NASC, those with dune points, and those that are oriented perpendicular to shore.
2. Use the __Copy Features__ geoprocessing tool to copy only the selected transects into a new feature class. If desired, here is the code that could be used to copy the selected features and clear the selection:
    ```python
    arcpy.CopyFeatures_management(extendedTrans, overlapTrans_lines)
    arcpy.SelectLayerByAttribute_management(extendedTrans, "CLEAR_SELECTION")
    ```
3. Run the code below to split the transects at the selected lines of overlap.

In [ ]:
overlapTrans_lines = os.path.join(arcpy.env.scratchGDB, overlapTrans_lines)
if not arcpy.Exists(overlapTrans_lines):
    overlapTrans_lines = input("Filename of the feature class of only 'boundary' transects: ")
arcpy.Intersect_analysis([extendedTrans, overlapTrans_lines], trans_x,
                         'ALL', output_type="POINT")
arcpy.SplitLineAtPoint_management(extendedTrans, trans_x, extTrans_tidy)

Delete the extraneous segments manually. Recommended:

1. Using __Select with Line__ draw a line to the appropriate side of the boundary transects. This will select the line segments that need to be deleted. 
2. __Delete__ the selected lines.
3. Remove any remaining overlaps entirely by hand. Use the __Split Line__ tool in the Editing toolbar to split lines to be shortened at the points of overlap. Then delete the remnant sections. 